In [1]:
# basin attributes will be used for large-sample emulator training
# all functions needed implement all basin emulator
import glob, os, sys, toml, pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

def read_camels_attributes():

    if True:

        # Load basin info
        infile_basin_info = f"/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv"
        df_basin_info = pd.read_csv(infile_basin_info)
    
        # Load basin attributes for this cluster
        attfiles = [
            "/glade/campaign/ral/hap/common/camels/camels_geol.txt",
            "/glade/campaign/ral/hap/common/camels/camels_hydro.txt",
            "/glade/campaign/ral/hap/common/camels/camels_clim.txt",
            "/glade/campaign/ral/hap/common/camels/camels_loc_topo.txt",
            "/glade/campaign/ral/hap/common/camels/camels_soil.txt",
            "/glade/campaign/ral/hap/common/camels/camels_vege.txt",
        ]
    
        for i in range(len(attfiles)):
            dfi = pd.read_csv(attfiles[i], delimiter=";")
            if i == 0:
                df_att = dfi
            else:
                df_att = pd.merge(df_att, dfi, on="gauge_id")
    
        df_att = df_att.loc[df_att["gauge_id"].isin(df_basin_info["hru_id"].values)]
        df_att.sel_index = np.arange(len(df_att))
        if np.any(df_att["gauge_id"].values != df_basin_info["hru_id"].values):
            sys.exit("Mismatch between att and info ids")
        else:
            print("att and info ids match")
            df_att["hru_id"] = df_basin_info["hru_id"].values

        
    return df_att


In [2]:
file = "/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_complete_attributes.csv"
if os.path.isfile(file):
    df_att = pd.read_csv(file)
else:
    df_att = read_camels_attributes()
    df_att.to_csv(file, index=False)

In [3]:
infile_attr_foruse = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/data/camels_attributes_table_TrainModel.csv'
df_att_foruse = pd.read_csv(infile_attr_foruse)
useattrs = list(df_att_foruse[df_att_foruse['att_Xie2021'].values]['Attribute_text'].values)
print("The number of attributes used:", len(useattrs))
print(useattrs)

The number of attributes used: 27
['mean_elev', 'mean_slope', 'area_gauges2', 'p_mean', 'pet_mean', 'aridity', 'p_seasonality', 'frac_snow', 'high_prec_freq', 'high_prec_dur', 'low_prec_freq', 'low_prec_dur', 'frac_forest', 'lai_max', 'lai_diff', 'dom_land_cover', 'dom_land_cover_frac', 'soil_depth_pelletier', 'soil_depth_statsgo', 'soil_porosity', 'soil_conductivity', 'max_water_content', 'sand_frac', 'silt_frac', 'clay_frac', 'carbonate_rocks_frac', 'geol_permeability']


In [16]:
    df_att_use = df_att[useattrs + ["hru_id"]]
    # One-hot encoding for categorical attributes
    for att in useattrs:
        if df_att_use[att].dtype == "object":
            print('Convert', att, 'to one-hot encoding')
            enc = OneHotEncoder(sparse=False)
            enc.fit(df_att_use[[att]])
            encnames = [att + "_" + str(i) for i in range(len(enc.categories_[0]))]
            print('New columns:', encnames)
            df_enc = pd.DataFrame(enc.transform(df_att_use[[att]]), columns=encnames)
            df_att_use = pd.concat([df_att_use, df_enc], axis=1)
            df_att_use = df_att_use.drop([att], axis=1)

    df_att_use.to_csv("/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_Xie2021_EmulatorTrain.csv", index=False)

Convert dom_land_cover to one-hot encoding
New columns: ['dom_land_cover_0', 'dom_land_cover_1', 'dom_land_cover_2', 'dom_land_cover_3', 'dom_land_cover_4', 'dom_land_cover_5', 'dom_land_cover_6', 'dom_land_cover_7', 'dom_land_cover_8', 'dom_land_cover_9', 'dom_land_cover_10', 'dom_land_cover_11']


/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [14]:
df_att_use

,mean_elev,mean_slope,area_gauges2,p_mean,pet_mean,aridity,p_seasonality,frac_snow,high_prec_freq,high_prec_dur,...,dom_land_cover_2,dom_land_cover_3,dom_land_cover_4,dom_land_cover_5,dom_land_cover_6,dom_land_cover_7,dom_land_cover_8,dom_land_cover_9,dom_land_cover_10,dom_land_cover_11
0,250.31,21.64152,2252.70,3.126679,1.971555,0.630559,0.187940,0.313440,12.95,1.348958,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,92.68,17.79072,573.60,3.608126,2.119256,0.587356,-0.114530,0.245259,20.55,1.205279,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,143.80,12.79195,3676.17,3.274405,2.043594,0.624111,0.047358,0.277018,17.15,1.207746,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,247.80,29.56035,769.05,3.522957,2.071324,0.587950,0.104091,0.291836,18.90,1.148936,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,310.38,49.92122,909.10,3.323146,2.090024,0.628929,0.147776,0.280118,20.10,1.165217,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,709.83,110.42527,224.92,4.977781,3.122204,0.627228,-0.995847,0.061255,15.10,1.776471,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
623,952.26,119.08920,587.90,4.543400,2.277630,0.501305,-0.821172,0.176337,14.75,1.446078,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
624,656.53,124.96889,443.07,6.297437,2.434652,0.386610,-0.952055,0.030203,14.60,1.467337,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
625,875.67,109.93127,41.42,2.781676,3.325188,1.195390,-0.985486,0.141500,20.45,1.786026,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
